In [1]:
import numpy as np

In [2]:
def createC1(dataSet):
    C1 = []
    for dS in dataSet:
        for item in dS:
            if [item] not in C1:
                C1.append([item])
    C1.sort()
    return map(frozenset, C1)
def scanD(D, Ck, minSupport):
    record = {}
    for d in D:
        for candidate in Ck:
            if candidate.issubset(d):
                if candidate not in record.keys():
                    record[candidate] = 1
                else:
                    record[candidate] += 1
    numItems = float(len(D))
    itemList = []
    supportData = {}
    for key in record:
        support = record[key] / numItems
        if support >= minSupport:
            itemList.append(key)
        supportData[key] = support
    return itemList, supportData

def generateCk(Lk, k):
    itemList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk):
            L1 = list(Lk[i])[:k-2]
            L2 = list(Lk[j])[:k-2]
            L1.sort()
            L2.sort()
            if L1 == L2:
                itemList.append(Lk[i] | Lk[j])
    return itemList

def apriori(dataSet, minSupport = 0.5):
    C1 = list(createC1(dataSet))
    D = list(map(set, dataSet))
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while(len(L[k-2])):
        Ck = generateCk(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData

def rules(L, supportData, minConf = 0.7):
    ruleList = []
    for i in range(1, len(L)):
        for freqSet in L[i]:
            H = [frozenset(item) for item in freqSet]
            if i == 1:
                calcConf(freqSet, H, supportData, ruleList, minConf)
            else:
                rulesFromConseq(freqSet, H, supportData, ruleList, minConf)
    return ruleList

def calcConf(freqSet, H, supportData, ruleList, minConf):
    prunedH = []
    for conseq in H:
        conf = supportData[freqSet] / supportData[freqSet - conseq]
        if conf >= minConf:
            print(freqSet - conseq, '-->', conseq, 'conf:', conf)
            ruleList.append(conseq)
            prunedH.append(conseq)
    return prunedH

def rulesFromConseq(freqSet, H, supportData, ruleList, minConf):
    m = len(H[0])
    if len(freqSet) > (m+1):
        Hmp1 = generateCk(H, m+1)
        Hmp1 = calcConf(freqSet, Hmp1, supportData, ruleList, minConf)
        if len(Hmp1) > 1:
            rulesFromConseq(freqSet, Hmp1, supportData, ruleList, minConf)

In [35]:
def loadDataSet():
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
    C1.sort()
    return map(frozenset, C1)
def scanD(D, Ck, minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck: #candidates
            if can.issubset(tid):
                if can not in ssCnt.keys():
                    ssCnt[can] = 1
                else:
                    ssCnt[can] += 1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key] / numItems
        if support >= minSupport:
            retList.insert(0, key)
        supportData[key] = support
    return retList, supportData
def aprioriGen(Lk, k):
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk):
            L1 = list(Lk[i])[:k-2]
            L2 = list(Lk[j])[:k-2]
            L1.sort()
            L2.sort()
            if L1 == L2:
                retList.append(Lk[i] | Lk[j]) #union
    return retList
def apriori(dataSet, minSupport = 0.5):
    C1 = list(createC1(dataSet))
    D = list(map(set, dataSet))
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while(len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData
def generateRules(L, supportData, minConf = 0.7):
    bigRuleList = []
    for i in range(1, len(L)):
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if i > 1:
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList
def calcConf(freqSet, H, supportData, brl, minConf = 0.7):
    prunedH = []
    for conseq in H:
        conf = supportData[freqSet] / supportData[freqSet - conseq]
        if conf >= minConf:
            print(freqSet-conseq, '-->', conseq, 'conf:', conf)
            brl.append(conseq)
            prunedH.append(conseq)
    return prunedH
def rulesFromConseq(freqSet, H, supportData, brl, minConf = 0.7):
    m = len(H[0])
    if len(freqSet) > (m + 1):
        Hmp1 = aprioriGen(H, m + 1)
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        if len(Hmp1) > 1:
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)


In [51]:
dataSet = loadDataSet()
L, supportData = apriori(dataSet)
rules = generateRules(L, supportData, minConf = 0.6)

frozenset({3}) --> frozenset({1}) conf: 0.6666666666666666
frozenset({1}) --> frozenset({3}) conf: 1.0
frozenset({3}) --> frozenset({2}) conf: 0.6666666666666666
frozenset({2}) --> frozenset({3}) conf: 0.6666666666666666
frozenset({5}) --> frozenset({3}) conf: 0.6666666666666666
frozenset({3}) --> frozenset({5}) conf: 0.6666666666666666
frozenset({5}) --> frozenset({2}) conf: 1.0
frozenset({2}) --> frozenset({5}) conf: 1.0
frozenset({5}) --> frozenset({2, 3}) conf: 0.6666666666666666
frozenset({3}) --> frozenset({2, 5}) conf: 0.6666666666666666
frozenset({2}) --> frozenset({3, 5}) conf: 0.6666666666666666


In [43]:
L, supportData = apriori(np.loadtxt('mushroom.dat').astype(int).astype(str), 0.4)

In [44]:
for item in L[1]:
    if item.intersection('2'):
        print(item)

frozenset({'2', '34'})
frozenset({'2', '59'})
frozenset({'63', '2'})
frozenset({'2', '85'})
frozenset({'2', '86'})
frozenset({'2', '90'})
frozenset({'2', '39'})
frozenset({'2', '28'})
